# 那些功能逆天，却鲜为人知的 pandas 骚操作
[链接](https://mp.weixin.qq.com/s/hUSWszONjdhv9jmmM0zDgQ)

## ACCESSOR
pandas有一种功能非常强大的方法，它就是accessor，可以将它理解为一种属性接口，通过它可以获得额外的方法。其实这样说还是很笼统，下面我们通过代码和实例来理解一下。

In [1]:
import pandas as pd

In [2]:
pd.Series._accessors

{'cat', 'dt', 'sparse', 'str'}

对于Series数据结构使用_accessors方法，可以得到了3个对象：cat，str，dt。

- .cat：用于分类数据（Categorical data）
- .str：用于字符数据（String Object data）
- .dt：用于时间数据（datetime-like data）
- .sparse：原文没有，待探索

下面我们依次看一下这四个对象是如何使用的。

### str对象的使用

In [3]:
# Series数据类型：str字符串
addr = pd.Series([
     'Washington, D.C. 20003',
     'Brooklyn, NY 11211-1755',
     'Omaha, NE 68154',
     'Pittsburgh, PA 15211'
]) 

In [4]:
addr.str.upper()

0     WASHINGTON, D.C. 20003
1    BROOKLYN, NY 11211-1755
2            OMAHA, NE 68154
3       PITTSBURGH, PA 15211
dtype: object

In [6]:
addr.str.lower()

0     washington, d.c. 20003
1    brooklyn, ny 11211-1755
2            omaha, ne 68154
3       pittsburgh, pa 15211
dtype: object

In [7]:
addr.str.count(r'\d')

0    5
1    9
2    5
3    5
dtype: int64

关于以上str对象的2个方法说明：
- Series.str.upper：将Series中所有字符串变为大写
- Series.str.count：对Series中所有字符串的个数进行计数

其实不难发现，该用法的使用与Python中字符串的操作很相似。没错，在pandas中你一样可以这样简单的操作，而不同的是你操作的是一整列的字符串数据。仍然基于以上数据集，再看它的另一个操作：

In [8]:
regex = (r'(?P<city>[A-Za-z ]+), '      # 一个或更多字母
         r'(?P<state>[A-Z]{2}) '        # 两个大写字母
         r'(?P<zip>\d{5}(?:-\d{4})?)')  # 可选的4个延伸数字

In [9]:
addr.str.replace('.', '').str.extract(regex)

,city,state,zip
0,Washington,DC,20003
1,Brooklyn,NY,11211-1755
2,Omaha,NE,68154
3,Pittsburgh,PA,15211


关于以上str对象的2个方法说明：
- Series.str.replace：将Series中指定字符串替换
- Series.str.extract：通过正则表达式提取字符串中的数据信息

这个用法就有点复杂了，因为很明显看到，这是一个链式的用法。通过replace将 " . " 替换为""，即为空，紧接着又使用了3个正则表达式（分别对应city，state，zip）通过extract对数据进行了提取，并由原来的Series数据结构变为了DataFrame数据结构。

In [11]:
# 当然，除了以上用法外，常用的属性和方法还有.rstrip，.contains，split等，我们通过下面代码查看一下str属性的完整列表：
[i for i in dir(pd.Series.str) if not i.startswith('_')]

['capitalize',
 'casefold',
 'cat',
 'center',
 'contains',
 'count',
 'decode',
 'encode',
 'endswith',
 'extract',
 'extractall',
 'find',
 'findall',
 'get',
 'get_dummies',
 'index',
 'isalnum',
 'isalpha',
 'isdecimal',
 'isdigit',
 'islower',
 'isnumeric',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'len',
 'ljust',
 'lower',
 'lstrip',
 'match',
 'normalize',
 'pad',
 'partition',
 'repeat',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'slice',
 'slice_replace',
 'split',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'wrap',
 'zfill']

### dt对象的使用
Series数据类型：datetime

因为数据需要datetime类型，所以下面使用pandas的date_range()生成了一组日期datetime演示如何进行dt对象操作。

In [21]:
daterng = pd.Series(pd.date_range('2017', periods=9, freq='Q'))

In [13]:
daterng

0   2017-03-31
1   2017-06-30
2   2017-09-30
3   2017-12-31
4   2018-03-31
5   2018-06-30
6   2018-09-30
7   2018-12-31
8   2019-03-31
dtype: datetime64[ns]

In [17]:
daterng = pd.Series(pd.date_range('2017', periods=40, freq='5D'))

In [18]:
daterng

0    2017-01-01
1    2017-01-06
2    2017-01-11
3    2017-01-16
4    2017-01-21
5    2017-01-26
6    2017-01-31
7    2017-02-05
8    2017-02-10
9    2017-02-15
10   2017-02-20
11   2017-02-25
12   2017-03-02
13   2017-03-07
14   2017-03-12
15   2017-03-17
16   2017-03-22
17   2017-03-27
18   2017-04-01
19   2017-04-06
20   2017-04-11
21   2017-04-16
22   2017-04-21
23   2017-04-26
24   2017-05-01
25   2017-05-06
26   2017-05-11
27   2017-05-16
28   2017-05-21
29   2017-05-26
30   2017-05-31
31   2017-06-05
32   2017-06-10
33   2017-06-15
34   2017-06-20
35   2017-06-25
36   2017-06-30
37   2017-07-05
38   2017-07-10
39   2017-07-15
dtype: datetime64[ns]

In [22]:
daterng.dt.day_name()

0      Friday
1      Friday
2    Saturday
3      Sunday
4    Saturday
5    Saturday
6      Sunday
7      Monday
8      Sunday
dtype: object

In [23]:
# 查看下半年
daterng[daterng.dt.quarter > 2]

2   2017-09-30
3   2017-12-31
6   2018-09-30
7   2018-12-31
dtype: datetime64[ns]

In [24]:
daterng[daterng.dt.is_year_end]

3   2017-12-31
7   2018-12-31
dtype: datetime64[ns]

以上关于dt的3种方法说明：
- Series.dt.day_name()：从日期判断出所处星期数
- Series.dt.quarter：从日期判断所处季节
- Series.dt.is_year_end：从日期判断是否处在年底
其它方法也都是基于datetime的一些变换，并通过变换来查看具体微观或者宏观日期。

In [25]:
# 当然，除了以上用法外，我们通过下面代码查看一下dt属性的完整列表：
[i for i in dir(pd.Series.dt) if not i.startswith('_')]

['asfreq',
 'ceil',
 'components',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days',
 'days_in_month',
 'daysinmonth',
 'end_time',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'microsecond',
 'microseconds',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'nanoseconds',
 'normalize',
 'quarter',
 'qyear',
 'round',
 'second',
 'seconds',
 'start_time',
 'strftime',
 'time',
 'timetz',
 'to_period',
 'to_pydatetime',
 'to_pytimedelta',
 'to_timestamp',
 'total_seconds',
 'tz',
 'tz_convert',
 'tz_localize',
 'week',
 'weekday',
 'weekofyear',
 'year']

### cat对象的使用
- Series数据类型：Category

在说cat对象的使用前，先说一下Category这个数据类型，它的作用很强大。虽然我们没有经常性的在内存中运行上g的数据，但是我们也总会遇到执行几行代码会等待很久的情况。使用Category数据的一个好处就是：可以很好的节省在时间和空间的消耗。下面我们通过几个实例来学习一下。

In [26]:
colors = pd.Series([
    'periwinkle',
    'mint green',
    'burnt orange',
    'periwinkle',
    'burnt orange',
    'rose',
    'rose',
    'mint green',
    'rose',
    'navy'
])

In [27]:
import sys

In [28]:
colors.apply(sys.getsizeof)

0    59
1    59
2    61
3    59
4    61
5    53
6    53
7    59
8    53
9    53
dtype: int64

上面我们通过使用sys.getsizeof来显示内存占用的情况，数字代表字节数。还有另一种计算内容占用的方法：memory_usage()，后面会使用。

现在我们将上面colors的不重复值映射为一组整数，然后再看一下占用的内存。

In [29]:
mapper = {v: k for k, v in enumerate(colors.unique())}

In [30]:
mapper

{'periwinkle': 0, 'mint green': 1, 'burnt orange': 2, 'rose': 3, 'navy': 4}

In [31]:
as_int = colors.map(mapper)

In [32]:
as_int

0    0
1    1
2    2
3    0
4    2
5    3
6    3
7    1
8    3
9    4
dtype: int64

In [33]:
as_int.apply(sys.getsizeof)

0    24
1    28
2    28
3    24
4    28
5    28
6    28
7    28
8    28
9    28
dtype: int64

注：对于以上的整数值映射也可以使用更简单的pd.factorize()方法代替。

我们发现上面所占用的内存是使用object类型时的一半。其实，这种情况就类似于Category data类型内部的原理。

`内存占用区别`：Categorical所占用的内存与Categorical分类的数量和数据的长度成正比，相反，object所占用的内存则是一个常数乘以数据的长度。

下面是object内存使用和category内存使用的情况对比。

In [34]:
colors.memory_usage(index=False, deep=True)

650

In [35]:
colors.astype('category').memory_usage(index=False, deep=True)

495

上面结果是使用object和Category两种情况下内存的占用情况。我们发现效果并没有我们想象中的那么好。但是注意Category内存是成比例的，如果数据集的数据量很大，但不重复分类（unique）值很少的情况下，`那么Category的内存占用可以节省达到10倍以上`，比如下面数据量增大的情况：

In [36]:
manycolors = colors.repeat(10)

In [37]:
len(manycolors)/manycolors.nunique()

20.0

In [38]:
manycolors.memory_usage(index=False, deep=True)

6500

In [39]:
manycolors.astype('category').memory_usage(index=False, deep=True)

585

可以看到，在数据量增加10倍以后，使用Category所占内容节省了10倍以上。

`除了占用内存节省外，另一个额外的好处是计算效率有了很大的提升`。因为对于Category类型的Series，str字符的操作发生在.cat.categories的非重复值上，而并非原Series上的所有元素上。也就是说对于每个非重复值都只做一次操作，然后再向与非重复值同类的值映射过去。

In [40]:
# 对于Category的数据类型，可以使用accessor的cat对象，以及相应的属性和方法来操作Category数据。
ccolors = colors.astype('category')

In [41]:
ccolors.cat.categories

Index(['burnt orange', 'mint green', 'navy', 'periwinkle', 'rose'], dtype='object')

In [42]:
# 实际上，对于开始的整数类型映射，可以先通过reorder_categories进行重新排序，然后再使用cat.codes来实现对整数的映射，来达到同样的效果。
ccolors.cat.reorder_categories(mapper).cat.codes

0    0
1    1
2    2
3    0
4    2
5    3
6    3
7    1
8    3
9    4
dtype: int8

dtype类型是Numpy的int8（-127~128）。可以看出以上只需要一个单字节就可以在内存中包含所有的值。我们开始的做法默认使用了int64类型，然而通过pandas的使用可以很智能的将Category数据类型变为最小的类型。

In [43]:
# 让我们来看一下cat还有什么其它的属性和方法可以使用。下面cat的这些属性基本都是关于查看和操作Category数据类型的。
[i for i in dir(ccolors.cat) if not i.startswith('_')]

['add_categories',
 'as_ordered',
 'as_unordered',
 'categories',
 'codes',
 'ordered',
 'remove_categories',
 'remove_unused_categories',
 'rename_categories',
 'reorder_categories',
 'set_categories']

In [44]:
# 但是Category数据的使用不是很灵活。例如，插入一个之前没有的值，首先需要将这个值添加到.categories的容器中，然后再添加值。
ccolors.iloc[5] = 'a new color'

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

In [45]:
ccolors = ccolors.cat.add_categories(['a new color'])

In [46]:
ccolors.iloc[5] = 'a new color'

In [47]:
ccolors

0      periwinkle
1      mint green
2    burnt orange
3      periwinkle
4    burnt orange
5     a new color
6            rose
7      mint green
8            rose
9            navy
dtype: category
Categories (6, object): [burnt orange, mint green, navy, periwinkle, rose, a new color]

如果你想设置值或重塑数据，而非进行新的运算操作，那么Category类型不是那么有用。

## 从clipboard剪切板载入数据

当我们的数据存在excel表里，或者其它的IDE编辑器中的时候，我们想要通过pandas载入数据。我们通常的做法是先保存再载入，其实这样做起来十分繁琐。一个简单的方法就是使用pd.read_clipboard() 直接从电脑的剪切板缓存区中提取数据。

这样我们就可以直接将结构数据转变为DataFrame或者Series了。excel表中数据是这样的：

https://mmbiz.qpic.cn/mmbiz_png/NOM5HN2icXzw038ysxxs1WTdR6YiaWIrphmnYEeZe926FbAUgShmawKwwndrMA4wz5NVfSibxNXCgj2cB5nVq1ybw/640?wx_fmt=png&wxfrom=5&wx_lazy=1&wx_co=1

## 使用"测试模块"制作伪数据

In [67]:
# 在pandas中，有一个测试模块可以帮助我们生成半真实（伪数据），并进行测试，它就是util.testing。下面同我们通过一个简单的例子看一下如何生成数据测试：
import pandas.util.testing as tm

In [68]:
# 默认的行和列
tm.N, tm.K = 15, 3

In [69]:
import numpy as np

In [70]:
np.random.seed(444)

In [71]:
tm.makeTimeDataFrame(freq='M').head()

,A,B,C,D
2000-01-31,0.357440,0.266873,0.353728,-0.536561
2000-02-29,0.377538,-0.480331,-0.433926,-0.886787
2000-03-31,1.382338,0.300781,-0.498028,0.107101
2000-04-30,1.175549,-0.179054,0.228771,-0.740890
2000-05-31,-0.939276,1.183669,-0.650078,-0.075697


In [72]:
tm.makeDataFrame().head()

,A,B,C,D
b8jgVbQbug,-0.748504,-0.099509,-0.060078,0.035310
OKCyyhkEvY,0.498427,0.798287,-0.169375,-1.487501
RtcTWq0AMT,-0.148212,0.507709,-0.089451,-0.716834
vtdamOujY0,-0.348742,0.273927,1.551892,-0.054453
tW49Zqe3lC,0.161808,0.839752,0.690683,1.536011


上面简单的使用了
`makeTimeDataFrame` 和 `makeDataFrame` 分别生成了一组时间数据和DataFrame的数据。

In [73]:
# 但这只是其中的两个用法，关于testing中的方法有大概30多个，如果你想全部了解，可以通过查看dir获得：
[i for i in dir(tm) if i.startswith('make')]

['makeBoolIndex',
 'makeCategoricalIndex',
 'makeCustomDataframe',
 'makeCustomIndex',
 'makeDataFrame',
 'makeDateIndex',
 'makeFloatIndex',
 'makeFloatSeries',
 'makeIntIndex',
 'makeIntervalIndex',
 'makeMissingCustomDataframe',
 'makeMissingDataframe',
 'makeMixedDataFrame',
 'makeMultiIndex',
 'makeObjectSeries',
 'makePeriodFrame',
 'makePeriodIndex',
 'makePeriodSeries',
 'makeRangeIndex',
 'makeStringIndex',
 'makeStringSeries',
 'makeTimeDataFrame',
 'makeTimeSeries',
 'makeTimedeltaIndex',
 'makeUIntIndex',
 'makeUnicodeIndex']

## 从列项中创建DatetimeIndex

In [53]:
# 也许我们有的时候会遇到这样的情形（为了说明这种情情况，我使用了product进行交叉迭代的创建了一组关于时间的数据）：
from itertools import product
import numpy as np

In [54]:
datecols = ['year', 'month', 'day']

In [55]:
df = pd.DataFrame(list(product([2017, 2016],[1, 2], [1, 2, 3])), columns=datecols)

In [56]:
df['data'] = np.random.randn(len(df))

In [57]:
df

,year,month,day,data
0,2017,1,1,-1.715649
1,2017,1,2,-1.157216
2,2017,1,3,-0.026555
3,2017,2,1,-0.134312
4,2017,2,2,-1.853022
5,2017,2,3,-0.152569
6,2016,1,1,1.165075
7,2016,1,2,0.387215
8,2016,1,3,-0.923056
9,2016,2,1,-2.010524


明显看到，列项中有`year，month，day`，它们分别在各个列中，而并非是一个完整日期。那么如何从这些列中将它们组合在一起并设置为新的index呢？

通过to_datetime的使用，我们就可以直接将年月日组合为一个完整的日期，然后赋给索引。代码如下：

In [58]:
df.index = pd.to_datetime(df[datecols])

In [59]:
df.head()

,year,month,day,data
2017-01-01,2017,1,1,-1.715649
2017-01-02,2017,1,2,-1.157216
2017-01-03,2017,1,3,-0.026555
2017-02-01,2017,2,1,-0.134312
2017-02-02,2017,2,2,-1.853022


In [60]:
# 当然，你可以选择将原有的年月日列移除，只保留data数据列，然后squeeze转换为Series结构。
df = df.drop(datecols, axis=1).squeeze()

In [61]:
df.head()

2017-01-01   -1.715649
2017-01-02   -1.157216
2017-01-03   -0.026555
2017-02-01   -0.134312
2017-02-02   -1.853022
Name: data, dtype: float64

In [65]:
df.index.dtype

dtype('<M8[ns]')

In [66]:
df.index.shape

(12,)